import pandas as pd
import matplotlib.pyplot as plt
# ^^^ pyforest auto-imports - don't write above this line


# Main Question: Do American teams have more American Players than Canadian Teams and vice versa for 2018-2019? 

## Imports

In [17]:
import warnings
from functions import *
import pickle
sns.set(style="whitegrid")

## Last Year Available

In [18]:
last_year_query = f"""
    SELECT DISTINCT(season)
    FROM game
"""
last_year_query_result = run_query(last_year_query)
last_year = last_year_query_result[-1][0]

## Canadian Teams List

### All teams plus their ID

In [19]:
# from game_team_stats table
all_teams_query = f"""
    SELECT COUNT(DISTINCT(team_id))
    FROM game_teams_stats
    INNER JOIN game
        ON game.game_id = game_teams_stats.game_id
    WHERE game.season = {last_year}
""" # adding the last year clause removed 3 teams
all_teams_query_result = run_query(all_teams_query)
all_teams_query_result

[(31,)]

### Canadian Team Ids 

In [20]:
# getting the actual team_id for each team
## also found here: https://www.kaggle.com/martinellis/nhl-game-data?select=team_info.csv
all_team_ids_query = f"""
    SELECT DISTINCT(team_id)
    FROM game_teams_stats
    INNER JOIN game
        ON game.game_id = game_teams_stats.game_id
    WHERE game.season = {last_year}
"""
all_team_ids_query_result = run_query(all_team_ids_query)

In [21]:
# flatten and sort
## https://stackoverflow.com/questions/952914/how-to-make-a-flat-list-out-of-list-of-lists
team_ids_flattened = [x for i in all_team_ids_query_result for x in i]
team_ids_sorted = sorted(team_ids_flattened)

In [22]:
canadian_team_dict = {'Vancouver': 26, 'Ottawa': 9,
                     'Montreal': 8, "Toronto": 10, "Calgary": 20,
                     "Winnipeg": 52, "Edmonton": 22}
canadian_team_ids = list(canadian_team_dict.values())
canadian_team_ids

[26, 9, 8, 10, 20, 52, 22]

## Combining all players into a dataframe

getting all teams, their last games, and their rosters for those games 


### All teams with their last game

In [23]:
last_game_roster_query = f"""
    SELECT team_id, MAX(gts.game_id)
    FROM game_teams_stats gts
    GROUP BY team_id
        HAVING MAX(gts.game_id) > 2018000000
"""
last_game_roster_query_result = run_query(last_game_roster_query)
len(last_game_roster_query_result)

31

### Getting and saving their rosters

In [24]:
players_in_last_game_roster_query = f"""
    SELECT team_id, game_id, pi.player_id, 
        pi.firstName, pi.lastName, pi.nationality, pi.birthCity, pi.primaryPosition
    FROM game_skater_stats gss
    INNER JOIN player_info pi
        ON gss.player_id = pi.player_id
    WHERE (team_id, game_id) IN
    (SELECT team_id, MAX(gts.game_id)
    FROM game_teams_stats gts
    GROUP BY team_id
        HAVING MAX(gts.game_id) > 2018000000
    )
"""
players_in_last_game_roster_query_result = run_query(players_in_last_game_roster_query)
len(players_in_last_game_roster_query_result)

849

In [25]:
# including players who didn't play in last game
length_of_player_info_table_query = f"""
    SELECT COUNT(DISTINCT(player_id))
    FROM player_info
"""
length_of_player_info_table_query_result = run_query(length_of_player_info_table_query)
length_of_player_info_table_query_result[0][0]

1205

In [26]:
# approx number of players in last game
num_of_teams = 31
players_per_team = 21
num_of_teams * players_per_team

651

#### Inputting players into dataframe

In [27]:
player_df = pd.DataFrame(data = players_in_last_game_roster_query_result,
                        columns = ['Team_id', "Game_id", "Player_id", "First_name",
                                  "Last_name", "Country_code", "Birth_city", "Primary_position"])

In [28]:
# only players, no goalie info
player_df.Primary_position.unique()

array(['C', 'D', 'RW', 'LW'], dtype=object)

In [29]:
# removing duplicates 
player_df.drop_duplicates(subset=['Player_id'], inplace=True)
player_df.reset_index(inplace=True, drop=True)
player_df.shape

(283, 8)

##### Adding Canadian Team Feature

In [30]:
# adding canada feature
canadian_team_values = list(canadian_team_dict.values())
canadian_team_values

[26, 9, 8, 10, 20, 52, 22]

In [31]:
in_canada = player_df['Team_id'].isin(canadian_team_values)
in_canada.sum()

62

In [32]:
player_df['Canadian_team'] = in_canada

In [36]:
# adding canadian and american player_feature
player_df['Canadian_player'] = player_df['Country_code'] == 'CAN'
player_df['Amer_player'] = player_df['Country_code'] == 'USA'
# adding dummy column so we know how many people are on each team when we use groupby
player_df['Dummy'] = 1
player_df.to_csv("player_df")

#### Grouping By Team

In [35]:
# groupby sum for each country_code and team
# lots of teams are missing players, so we will aggregate the canadian and american teams
player_df_groupby_sum = player_df.groupby('Team_id').sum()
player_df_groupby_sum.to_csv("player_df_groupby_sum")